# Album aggs and Song dataframe Updates based on Billboard Hot 100 Updates


## Based on newly released data (January 1st, 2023) and expanding whole career (vs. August 2019)

Prior dataframe only had top song flag and boolean value. This script will update with specific song counts based on Billboard charts. It will allow me to look for correlations within song characteristics and success measures. 

I am utilizing figures from Tableau visualizations and Billboard dataframe.

I already put all of the information about songs into the Billboard charts songs, but this new dataframe will allow for comparison of songs that make the Billboard charts vs. those that don't, and will allow clustering analysis to help identify trends.

After this script, I will conduct a K-means clustering analysis for song in a different notebook.

## Contents

## 01. Import libraries and song dataframe

## 02. Review data

## 03. Wrangling: 
- update top song counts
- merge songs df with subset of billboard chart week counts

## 04. Data export

# 01. Import libraries and data

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import os

In [2]:
# Create path

path = r"C:\Users\leobs\OneDrive\Desktop\LF_TS_Analysis"

In [3]:
# Import song stream for merging song info

df = pd.read_csv(os.path.join(path,'02 Manipulated Data', 'song_stream_alb_final_3.csv'), index_col = 0)

In [4]:
# Import billboard df

bill = pd.read_csv(os.path.join(path,'02 Manipulated Data', 'df_hot_100_ts_only_dec_23.csv'), index_col = 0)

# 02. Review data

In [5]:
df.head()

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,1,0.009420,0.757,0.610,0.000037,0.3670,-4.840,...,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
1,Blank Space (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,2,0.088500,0.733,0.733,0.000000,0.1680,-5.376,...,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
2,Style (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,3,0.000421,0.511,0.822,0.019700,0.0899,-4.785,...,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,4,0.000537,0.545,0.885,0.000056,0.3850,-5.968,...,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,5,0.000656,0.588,0.721,0.000000,0.1310,-5.579,...,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2


In [6]:
df.shape

(285, 43)

In [7]:
df.tail()

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
280,Our Song,Debut,10/24/2006,11,0.11100,0.668,0.672,0.000000,0.3290,-4.931,...,10/24/2006,39000,284,0.045622,15,False,Low first week sales,High total streams,Country pop,3
281,I'm Only Me When I'm With You,Debut,10/24/2006,12,0.00452,0.563,0.934,0.000807,0.1030,-3.629,...,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3
282,Invisible,Debut,10/24/2006,13,0.63700,0.612,0.394,0.000000,0.1470,-5.723,...,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3
283,A Perfectly Good Heart,Debut,10/24/2006,14,0.00349,0.483,0.751,0.000000,0.1280,-5.726,...,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3
284,Teardrops on My Guitar - Pop Version,Debut,10/24/2006,15,0.04020,0.459,0.753,0.000000,0.0863,-3.827,...,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285 entries, 0 to 284
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   song_title           285 non-null    object 
 1   album_title          285 non-null    object 
 2   song_release_date    285 non-null    object 
 3   track                285 non-null    int64  
 4   acou                 285 non-null    float64
 5   dance                285 non-null    float64
 6   ener                 285 non-null    float64
 7   instru               285 non-null    float64
 8   live                 285 non-null    float64
 9   loud                 285 non-null    float64
 10  speech               285 non-null    float64
 11  temp                 285 non-null    float64
 12  val                  285 non-null    float64
 13  pop                  285 non-null    int64  
 14  dur                  285 non-null    int64  
 15  album_id             285 non-null    object 


In [9]:
df.columns

Index(['song_title', 'album_title', 'song_release_date', 'track', 'acou',
       'dance', 'ener', 'instru', 'live', 'loud', 'speech', 'temp', 'val',
       'pop', 'dur', 'album_id', 'song_id', 'days', 'tot_streams',
       'last_stream_update', 'streams_n', 'top_song', 'top_song_flag',
       'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo', 'alb_ann_day',
       'alb_ann_yr', 'album_announce_date', 'album_release_mo', 'alb_r_day',
       'alb_r_yr', 'album_release_date', 'w1_sales', 'top_alb_count',
       'top_alb_normal', 'alb_track_cnt', 'tvflag', 'sales_group',
       'tot_streams_group', 'alb_genre', 'cluster'],
      dtype='object')

In [10]:
bill.head()

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
884.0,7/27/2013,65.0,22,Taylor Swift,55.0,20.0,20.0,Red (Deluxe Edition),10/22/2012,6.0,...,10/22/2012,1208000.0,185.0,0.045849,22.0,False,High first week sales,High total streams,Pop rock,both
885.0,7/20/2013,55.0,22,Taylor Swift,49.0,20.0,19.0,Red (Deluxe Edition),10/22/2012,6.0,...,10/22/2012,1208000.0,185.0,0.045849,22.0,False,High first week sales,High total streams,Pop rock,both
886.0,7/13/2013,49.0,22,Taylor Swift,41.0,20.0,18.0,Red (Deluxe Edition),10/22/2012,6.0,...,10/22/2012,1208000.0,185.0,0.045849,22.0,False,High first week sales,High total streams,Pop rock,both
887.0,7/6/2013,41.0,22,Taylor Swift,32.0,20.0,17.0,Red (Deluxe Edition),10/22/2012,6.0,...,10/22/2012,1208000.0,185.0,0.045849,22.0,False,High first week sales,High total streams,Pop rock,both
889.0,6/29/2013,32.0,22,Taylor Swift,26.0,20.0,16.0,Red (Deluxe Edition),10/22/2012,6.0,...,10/22/2012,1208000.0,185.0,0.045849,22.0,False,High first week sales,High total streams,Pop rock,both


In [11]:
bill.shape

(1446, 49)

In [12]:
bill.tail()

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
251.0,12/3/2022,67.0,"You're On Your Own, Kid",Taylor Swift,48.0,8.0,5.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,...,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
259.0,11/26/2022,48.0,"You're On Your Own, Kid",Taylor Swift,42.0,8.0,4.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,...,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
273.0,11/19/2022,42.0,"You're On Your Own, Kid",Taylor Swift,20.0,8.0,3.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,...,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
297.0,11/12/2022,20.0,"You're On Your Own, Kid",Taylor Swift,8.0,8.0,2.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,...,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
318.0,11/5/2022,8.0,"You're On Your Own, Kid",Taylor Swift,0.0,8.0,1.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,...,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both


In [13]:
bill.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1446 entries, 884.0 to 318.0
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   chart_week           1446 non-null   object 
 1   chart_position       1446 non-null   float64
 2   song_title           1446 non-null   object 
 3   artist               1446 non-null   object 
 4   prior_wk_position    1446 non-null   float64
 5   peak_pos             1446 non-null   float64
 6   wks_on_chart         1446 non-null   float64
 7   album_title          1446 non-null   object 
 8   song_release_date    1446 non-null   object 
 9   track                1446 non-null   float64
 10  acou                 1446 non-null   float64
 11  dance                1446 non-null   float64
 12  ener                 1446 non-null   float64
 13  instru               1446 non-null   float64
 14  live                 1446 non-null   float64
 15  loud                 1446 non-null   f

In [14]:
bill['song_id'].value_counts(dropna=False)

song_id
MIDN03      53
FEAR12      50
198906      50
FEAR09      49
DEBUT03     48
            ..
REDTV11      1
1989TV10     1
RED07        1
FOLK15       1
LOVE07       1
Name: count, Length: 214, dtype: int64

In [15]:
bill['song_id'].value_counts(dropna=False)

song_id
MIDN03      53
FEAR12      50
198906      50
FEAR09      49
DEBUT03     48
            ..
REDTV11      1
1989TV10     1
RED07        1
FOLK15       1
LOVE07       1
Name: count, Length: 214, dtype: int64

There is a slight mismatch- 214 billboard songs vs. 216 top songs listed. Will address during wrangling.

# 03. Wrangling

## Update removal of voice memos (previous issue)
- 3 songs from 1989 are voice memos and should be removed

In [17]:
df.shape

(285, 43)

In [22]:
check_1989 = df.loc[df['album_id'] == "1989"]

In [23]:
check_1989

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
190,Welcome To New York,1989 (Deluxe Edition),10/27/2014,1,0.034800,0.789,0.634,0.000002,0.3020,-4.762,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
191,Blank Space,1989 (Deluxe Edition),10/27/2014,2,0.103000,0.760,0.703,0.000000,0.0913,-5.412,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
192,Style,1989 (Deluxe Edition),10/27/2014,3,0.002450,0.588,0.791,0.002580,0.1180,-5.595,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
193,Out Of The Woods,1989 (Deluxe Edition),10/27/2014,4,0.000743,0.553,0.841,0.000012,0.3410,-6.937,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
194,All You Had To Do Was Stay,1989 (Deluxe Edition),10/27/2014,5,0.002010,0.605,0.725,0.000020,0.1010,-5.729,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
195,Shake It Off,1989 (Deluxe Edition),10/27/2014,6,0.064700,0.647,0.800,0.000000,0.3340,-5.384,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
196,I Wish You Would,1989 (Deluxe Edition),10/27/2014,7,0.015800,0.653,0.893,0.000049,0.1020,-5.966,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
197,Bad Blood,1989 (Deluxe Edition),10/27/2014,8,0.088500,0.646,0.794,0.000006,0.2010,-6.104,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
198,Wildest Dreams,1989 (Deluxe Edition),10/27/2014,9,0.069200,0.550,0.688,0.001440,0.1060,-7.416,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
199,How You Get The Girl,1989 (Deluxe Edition),10/27/2014,10,0.003640,0.765,0.656,0.008680,0.0918,-6.112,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4


In [19]:
# Define songs that are voice memos to remove

songs_to_remove = ["198917", "198918", "198919"]

In [24]:
# Filter rows in updated dataframe to not include those three songs
# choosing to keep same name (df) due to other manipulation in scripts

df = df[~df['song_id'].isin(songs_to_remove)]

In [25]:
df.shape

(282, 43)

In [26]:
# Re-define check to see if 1989 album is updated without 3 songs

check_1989 = df.loc[df['album_id'] == "1989"]

In [27]:
check_1989

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,...,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
190,Welcome To New York,1989 (Deluxe Edition),10/27/2014,1,0.034800,0.789,0.634,0.000002,0.3020,-4.762,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
191,Blank Space,1989 (Deluxe Edition),10/27/2014,2,0.103000,0.760,0.703,0.000000,0.0913,-5.412,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
192,Style,1989 (Deluxe Edition),10/27/2014,3,0.002450,0.588,0.791,0.002580,0.1180,-5.595,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
193,Out Of The Woods,1989 (Deluxe Edition),10/27/2014,4,0.000743,0.553,0.841,0.000012,0.3410,-6.937,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
194,All You Had To Do Was Stay,1989 (Deluxe Edition),10/27/2014,5,0.002010,0.605,0.725,0.000020,0.1010,-5.729,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
195,Shake It Off,1989 (Deluxe Edition),10/27/2014,6,0.064700,0.647,0.800,0.000000,0.3340,-5.384,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
196,I Wish You Would,1989 (Deluxe Edition),10/27/2014,7,0.015800,0.653,0.893,0.000049,0.1020,-5.966,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4
197,Bad Blood,1989 (Deluxe Edition),10/27/2014,8,0.088500,0.646,0.794,0.000006,0.2010,-6.104,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
198,Wildest Dreams,1989 (Deluxe Edition),10/27/2014,9,0.069200,0.550,0.688,0.001440,0.1060,-7.416,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,High total streams,Synth/Dream/Dance pop,4
199,How You Get The Girl,1989 (Deluxe Edition),10/27/2014,10,0.003640,0.765,0.656,0.008680,0.0918,-6.112,...,11/15/2014,1297000,464,0.140606,19,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,4


## Updating with billboard data
- Will use billboard df count of chart weeks to add count of weeks

In [28]:
song_count = bill['song_id'].value_counts().reset_index()

In [29]:
song_count.head()

,song_id,count
0,MIDN03,53
1,FEAR12,50
2,198906,50
3,FEAR09,49
4,DEBUT03,48


In [30]:
song_count.columns = ['song_id', 'cnt_chart_wks']

In [31]:
song_count.head()

,song_id,cnt_chart_wks
0,MIDN03,53
1,FEAR12,50
2,198906,50
3,FEAR09,49
4,DEBUT03,48


In [32]:
song_count.shape

(214, 2)

## Apply information to song dataframe


In [33]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

### Merge song_count dataframe with songs
### Any song without a merge flag should be filled in with "0" for count

In [34]:
# Merge dataframes

df_merged = df.merge(song_count, on= 'song_id', how='left', indicator=True)

In [35]:
df_merged.columns

Index(['song_title', 'album_title', 'song_release_date', 'track', 'acou',
       'dance', 'ener', 'instru', 'live', 'loud', 'speech', 'temp', 'val',
       'pop', 'dur', 'album_id', 'song_id', 'days', 'tot_streams',
       'last_stream_update', 'streams_n', 'top_song', 'top_song_flag',
       'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo', 'alb_ann_day',
       'alb_ann_yr', 'album_announce_date', 'album_release_mo', 'alb_r_day',
       'alb_r_yr', 'album_release_date', 'w1_sales', 'top_alb_count',
       'top_alb_normal', 'alb_track_cnt', 'tvflag', 'sales_group',
       'tot_streams_group', 'alb_genre', 'cluster', 'cnt_chart_wks', '_merge'],
      dtype='object')

In [36]:
df_merged['_merge'].value_counts(dropna=False)

_merge
both          214
left_only      68
right_only      0
Name: count, dtype: int64

In [37]:
df_merged['cnt_chart_wks'].value_counts(dropna=False)

cnt_chart_wks
1.0     98
NaN     68
2.0     32
3.0     15
20.0    11
5.0      8
4.0      5
8.0      5
6.0      4
22.0     3
19.0     3
15.0     3
13.0     2
21.0     2
36.0     2
50.0     2
7.0      2
10.0     1
23.0     1
12.0     1
49.0     1
28.0     1
25.0     1
24.0     1
14.0     1
32.0     1
38.0     1
35.0     1
33.0     1
11.0     1
9.0      1
53.0     1
29.0     1
48.0     1
Name: count, dtype: int64

In [38]:
# Check for the songs not added

song_check = df_merged[(df_merged['top_song'] == "top song") & pd.isna(df_merged['cnt_chart_wks'])]

In [39]:
pd.set_option('display.max_columns', None)

In [40]:
song_check

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster,cnt_chart_wks,_merge


## Checked sources to address each song

Billboard df changes: 
- Bad Blood(feat. Kendrick Lamar)(Taylor's Version) is listed as 1989TV08 should have song ID 1989TV22
- Karma(feat. Ice Spice) is listed as song ID MIDN11 but should be MIDN23
- All Too Well (10 Minuted Version)(Taylor's Version) is mixed up. It should be the regular (not 10 min) version and have the id REDTV05 but should be listed as REDTV30

#### Update songs dataframe to list song ID SPEAK18 as not top

In [41]:
df_merged.columns

Index(['song_title', 'album_title', 'song_release_date', 'track', 'acou',
       'dance', 'ener', 'instru', 'live', 'loud', 'speech', 'temp', 'val',
       'pop', 'dur', 'album_id', 'song_id', 'days', 'tot_streams',
       'last_stream_update', 'streams_n', 'top_song', 'top_song_flag',
       'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo', 'alb_ann_day',
       'alb_ann_yr', 'album_announce_date', 'album_release_mo', 'alb_r_day',
       'alb_r_yr', 'album_release_date', 'w1_sales', 'top_alb_count',
       'top_alb_normal', 'alb_track_cnt', 'tvflag', 'sales_group',
       'tot_streams_group', 'alb_genre', 'cluster', 'cnt_chart_wks', '_merge'],
      dtype='object')

In [42]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [43]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [44]:
# For loop to edit SPEAK18

df.loc[df['song_id']== "SPEAK18", 'top_song'] = "not top"

In [45]:
# For loop to edit SPEAK18

df_merged.loc[df_merged['song_id']== "SPEAK18", 'top_song'] = "not top"

In [46]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [47]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [48]:
df_merged['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [49]:
df['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [50]:
# For loop to edit SPEAK18

df.loc[df['song_id']== "SPEAK18", 'top_song_flag'] = False

In [51]:
# For loop to edit SPEAK18

df_merged.loc[df_merged['song_id']== "SPEAK18", 'top_song_flag'] = False

In [52]:
df['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [53]:
df_merged['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [54]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [55]:
# Check All Too Well (both Taylor's versions)

ATW10 = df[df['song_id'] == "REDTV30"]

In [56]:
ATW10

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
96,All Too Well (10 Minute Version) (Taylor's Ver...,Red (Taylor's Version),11/12/2021,30,0.274,0.631,0.518,0.0,0.088,-8.771,0.0303,93.023,0.205,87,613026,REDTV,REDTV30,727,43596170,11/9/2023,59967,not top,False,26,Taylor's Version,11,June,18,2021,6/18/2021,November,12,2021,11/12/2021,369000,103,0.141678,30,True,Low first week sales,Low total streams,Pop rock,2


In [57]:
ATW = df[df['song_id'] == "REDTV05"]

In [58]:
ATW

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
71,All Too Well (Taylor's Version),Red (Taylor's Version),11/12/2021,5,0.0171,0.44,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.132,78,329160,REDTV,REDTV05,727,57826282,11/9/2023,79540,top song,True,26,Taylor's Version,11,June,18,2021,6/18/2021,November,12,2021,11/12/2021,369000,103,0.141678,30,True,Low first week sales,Medium total streams,Pop rock,2


In [59]:
# Value counts then for loop in both songs and merged df to update 
# 10 Minute version of All Too Well as Not a Top song

df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [60]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [61]:
# For loop to edit All Too Well, REDTV30

df.loc[df['song_id']== "REDTV30", 'top_song'] = "not top"
df_merged.loc[df_merged['song_id']== "REDTV30", 'top_song'] = "not top"

In [62]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [63]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [64]:
df['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [65]:
df_merged['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [66]:
# For loop to edit All Too Well, REDTV30 as top song flag

df.loc[df['song_id']== "REDTV30", 'top_song_flag'] = False
df_merged.loc[df_merged['song_id']== "REDTV30", 'top_song_flag'] = False

In [67]:
df['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [68]:
df_merged['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

### Need to update billboards information for songs for issues above

song ids: 
1989TV08 and 1989TV22
MIDN11 and MIDN23

In [69]:
# Locate relevant song ids

bill_edit = bill.loc[bill['song_id'].isin(["1989TV08", "1989TV22", "MIDN11", "MIDN23", "REDTV05", "REDTV30"])]

In [70]:
bill_edit

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
327.0,3/5/2022,99.0,All Too Well (Taylor's Version),Taylor Swift,96.0,1.0,15.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
328.0,2/26/2022,96.0,All Too Well (Taylor's Version),Taylor Swift,84.0,1.0,14.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
329.0,2/19/2022,84.0,All Too Well (Taylor's Version),Taylor Swift,75.0,1.0,13.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
331.0,2/12/2022,75.0,All Too Well (Taylor's Version),Taylor Swift,77.0,1.0,12.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
333.0,2/5/2022,77.0,All Too Well (Taylor's Version),Taylor Swift,74.0,1.0,11.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
336.0,1/29/2022,74.0,All Too Well (Taylor's Version),Taylor Swift,82.0,1.0,10.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
337.0,1/22/2022,82.0,All Too Well (Taylor's Version),Taylor Swift,55.0,1.0,9.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
340.0,1/15/2022,55.0,All Too Well (Taylor's Version),Taylor Swift,66.0,1.0,8.0,Red (Taylor's Version),11/12/2021,5.0,0.0171,0.440,0.528,0.00203,0.234,-7.809,0.0317,185.972,0.1320,78.0,329160.0,REDTV,REDTV05,727.0,57826282.0,11/9/2023,79540.0,top song,True,11.0,Taylor's Version,11.0,June,18.0,2021.0,6/18/2021,November,12.0,2021.0,11/12/2021,369000.0,103.0,0.141678,30.0,True,Low first week sales,Medium total streams,Pop rock,both
342.0,1/8/2022,66.0,All Too Well 

In [71]:
df.head()

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,1,0.009420,0.757,0.610,0.000037,0.3670,-4.840,0.0327,116.998,0.685,80,212600,1989TV,1989TV01,13,35521714,11/9/2023,2732439,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
1,Blank Space (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,2,0.088500,0.733,0.733,0.000000,0.1680,-5.376,0.0670,96.057,0.701,80,231833,1989TV,1989TV02,13,40958068,11/9/2023,3150620,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
2,Style (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,3,0.000421,0.511,0.822,0.019700,0.0899,-4.785,0.0397,94.868,0.305,81,231000,1989TV,1989TV03,13,47687102,11/9/2023,3668238,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,4,0.000537,0.545,0.885,0.000056,0.3850,-5.968,0.0447,92.021,0.206,80,235800,1989TV,1989TV04,13,37545128,11/9/2023,2888086,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,5,0.000656,0.588,0.721,0.000000,0.1310,-5.579,0.0317,96.997,0.520,79,193289,1989TV,1989TV05,13,31008534,11/9/2023,2385271,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2


In [72]:
# See if top song accurate- Bad Blood should be on chart, Bad Blood featuring 
bad_blood = df['album_id'] == "1989TV"

In [73]:
bad_blood_df = df[bad_blood]

In [74]:
bad_blood_df

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,1,0.009420,0.757,0.610,0.000037,0.3670,-4.840,0.0327,116.998,0.6850,80,212600,1989TV,1989TV01,13,35521714,11/9/2023,2732439,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
1,Blank Space (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,2,0.088500,0.733,0.733,0.000000,0.1680,-5.376,0.0670,96.057,0.7010,80,231833,1989TV,1989TV02,13,40958068,11/9/2023,3150620,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
2,Style (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,3,0.000421,0.511,0.822,0.019700,0.0899,-4.785,0.0397,94.868,0.3050,81,231000,1989TV,1989TV03,13,47687102,11/9/2023,3668238,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,4,0.000537,0.545,0.885,0.000056,0.3850,-5.968,0.0447,92.021,0.2060,80,235800,1989TV,1989TV04,13,37545128,11/9/2023,2888086,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,5,0.000656,0.588,0.721,0.000000,0.1310,-5.579,0.0317,96.997,0.5200,79,193289,1989TV,1989TV05,13,31008534,11/9/2023,2385271,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
5,Shake It Off (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,6,0.012100,0.636,0.808,0.000022,0.3590,-5.693,0.0729,160.058,0.9170,79,219209,1989TV,1989TV06,13,29684734,11/9/2023,2283441,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
6,I Wish You Would (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,7,0.003540,0.670,0.858,0.000013,0.0687,-6.528,0.0439,118.009,0.5390,79,207650,1989TV,1989TV07,13,27727454,11/9/2023,2132881,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
7,Bad Blood (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,8,0.036200,0.618,0.683,0.000000,0.3050,-6.438,0.1940,169.971,0.3630,78,211103,1989TV,1989TV08,13,26353077,11/9/2023,2027159,top song,False,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
8,Wildest Dreams (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,9,0.043600,0.589,0.674,0.000072,0.1120,-7.480,0.0656,139.985,0.5140,78,220433,1989TV,1989TV09,13,543373389,11/9/2023,41797953,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,High total streams,Synth/Dream/Dance pop,2
9,How You Get The Girl (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,10,0.001960,0.758,0.691,0.000011,0.0939,-5.798,0.

In [75]:
# Update song and merged df so that Bad Blood(feat. Kendrick Lamar)(Taylor's Version) is not a top song
# get counts

df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [76]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [77]:
# For loop to edit All Too Well, REDTV30

df.loc[df['song_id']== "1989TV22", 'top_song'] = "not top"
df_merged.loc[df_merged['song_id']== "1989TV22", 'top_song'] = "not top"

In [78]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [79]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [80]:
# For loop to edit All Too Well, REDTV30 as top song flag

df.loc[df['song_id']== "1989TV22", 'top_song_flag'] = False
df_merged.loc[df_merged['song_id']== "1989TV22", 'top_song_flag'] = False

In [81]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [82]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [83]:
bill.columns

Index(['chart_week', 'chart_position', 'song_title', 'artist',
       'prior_wk_position', 'peak_pos', 'wks_on_chart', 'album_title',
       'song_release_date', 'track', 'acou', 'dance', 'ener', 'instru', 'live',
       'loud', 'speech', 'temp', 'val', 'pop', 'dur', 'album_id', 'song_id',
       'days', 'tot_streams', 'last_stream_update', 'streams_n', 'top_song',
       'top_song_flag', 'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo',
       'alb_ann_day', 'alb_ann_yr', 'album_announce_date', 'album_release_mo',
       'alb_r_day', 'alb_r_yr', 'album_release_date', 'w1_sales',
       'top_alb_count', 'top_alb_normal', 'alb_track_cnt', 'tvflag',
       'sales_group', 'tot_streams_group', 'alb_genre', '_merge'],
      dtype='object')

In [84]:
# Update song id in Billboard df
# Bad Blood(feat. Kendrick Lamar)(Taylor's Version) is listed as 1989TV08 should have song ID 1989TV22
df.loc[df['song_id']== "1989TV08", 'top_song_flag'] = False

In [85]:
# Check information for billboard song dataframe

# Locate relevant song ids

bill_bb = bill.loc[bill['song_id'].isin(["1989TV08", "1989TV22"])]

In [86]:
bill_bb

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
22.0,11/25/2023,80.0,Bad Blood (Taylor's Version),Taylor Swift,47.0,7.0,3.0,1989 (Taylor's Version)[Deluxe],10/27/2023,8.0,0.0362,0.618,0.683,0.0,0.305,-6.438,0.194,169.971,0.363,78.0,211103.0,1989TV,1989TV08,13.0,26353077.0,11/9/2023,2027159.0,top song,True,14.0,Taylor's Version,14.0,August,9.0,2023.0,8/9/2023,October,27.0,2023.0,10/27/2023,1359000.0,1.0,0.076923,22.0,True,High first week sales,Low total streams,Synth/Dream/Dance pop,both
31.0,11/18/2023,47.0,Bad Blood (Taylor's Version),Taylor Swift,7.0,7.0,2.0,1989 (Taylor's Version)[Deluxe],10/27/2023,8.0,0.0362,0.618,0.683,0.0,0.305,-6.438,0.194,169.971,0.363,78.0,211103.0,1989TV,1989TV08,13.0,26353077.0,11/9/2023,2027159.0,top song,True,14.0,Taylor's Version,14.0,August,9.0,2023.0,8/9/2023,October,27.0,2023.0,10/27/2023,1359000.0,1.0,0.076923,22.0,True,High first week sales,Low total streams,Synth/Dream/Dance pop,both
60.0,11/11/2023,7.0,Bad Blood (Taylor's Version),Taylor Swift,0.0,7.0,1.0,1989 (Taylor's Version)[Deluxe],10/27/2023,8.0,0.0362,0.618,0.683,0.0,0.305,-6.438,0.194,169.971,0.363,78.0,211103.0,1989TV,1989TV08,13.0,26353077.0,11/9/2023,2027159.0,top song,True,14.0,Taylor's Version,14.0,August,9.0,2023.0,8/9/2023,October,27.0,2023.0,10/27/2023,1359000.0,1.0,0.076923,22.0,True,High first week sales,Low total streams,Synth/Dream/Dance pop,both


Accurate, no need to shift anything else

In [87]:
# Update song id in Billboard df

#Karma(feat. Ice Spice) is listed as song ID MIDN11 but should be MIDN23

bill_karma = bill.loc[bill['song_id'].isin(["MIDN11", "MIDN23"])]

In [88]:
bill_karma

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
73.0,10/14/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,31.0,2.0,30.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
76.0,10/7/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,38.0,2.0,29.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
80.0,9/30/2023,38.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,36.0,2.0,28.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
82.0,9/23/2023,36.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,24.0,2.0,27.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
85.0,9/16/2023,24.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,25.0,2.0,26.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
89.0,9/9/2023,25.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,15.0,2.0,25.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
92.0,9/2/2023,15.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,14.0,2.0,24.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
97.0,8/26/2023,14.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,13.0,2.0,23.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN2

In [89]:
bill.loc[bill['song_title']== "Karma (feat. Ice Spice)", 'song_id'] = "MIDN23"

In [90]:
bill_karma = bill.loc[bill['song_id'].isin(["MIDN11", "MIDN23"])]

In [91]:
bill_karma

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
73.0,10/14/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,31.0,2.0,30.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
76.0,10/7/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,38.0,2.0,29.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
80.0,9/30/2023,38.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,36.0,2.0,28.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
82.0,9/23/2023,36.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,24.0,2.0,27.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
85.0,9/16/2023,24.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,25.0,2.0,26.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
89.0,9/9/2023,25.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,15.0,2.0,25.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
92.0,9/2/2023,15.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,14.0,2.0,24.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
97.0,8/26/2023,14.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,13.0,2.0,23.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN2

In [92]:
# Update all of the song qualities and characteristics 
# for the Billboard charts for Karma(Featuring Ice Spice), based on song dataframe for MIDN23

karma_ice = df.loc[df["song_id"] == "MIDN23"]

In [93]:
karma_ice

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
66,Karma (feat. Ice Spice),Midnights (The Til Dawn Edition),5/26/2023,23,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75,201493,MIDN,MIDN23,187,67922508,11/9/2023,363221,top song,True,22,original,12,August,28,2022,8/28/2022,October,21,2022,10/21/2022,1140000,54,0.28877,23,False,High first week sales,Medium total streams,Synth/Dream/Dance pop,2


In [94]:
# Overwrite information, select rows to edit from df

condition = bill['song_id'] == "MIDN23"

In [95]:
karma_ice.columns

Index(['song_title', 'album_title', 'song_release_date', 'track', 'acou',
       'dance', 'ener', 'instru', 'live', 'loud', 'speech', 'temp', 'val',
       'pop', 'dur', 'album_id', 'song_id', 'days', 'tot_streams',
       'last_stream_update', 'streams_n', 'top_song', 'top_song_flag',
       'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo', 'alb_ann_day',
       'alb_ann_yr', 'album_announce_date', 'album_release_mo', 'alb_r_day',
       'alb_r_yr', 'album_release_date', 'w1_sales', 'top_alb_count',
       'top_alb_normal', 'alb_track_cnt', 'tvflag', 'sales_group',
       'tot_streams_group', 'alb_genre', 'cluster'],
      dtype='object')

In [96]:
new_karma_ice_values = {'song_release_date': 5/26/2023, 
                        'track': 23, 
                        'acou': 0.0623,
                       'dance': 0.615, 
                        'ener': 0.616, 
                        'instru': 0.0, 
                        'live': 0.579, 
                        'loud': -6.523, 
                        'speech': 0.0622, 
                        'temp': 89.977, 
                        'val': 0.0734,
                         'pop': 75, 
                        'dur': 201493, 
                        'days': 187, 
                        'tot_streams':67922508,
                         'streams_n': 363221}

In [97]:
bill_karma

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
73.0,10/14/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,31.0,2.0,30.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
76.0,10/7/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,38.0,2.0,29.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
80.0,9/30/2023,38.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,36.0,2.0,28.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
82.0,9/23/2023,36.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,24.0,2.0,27.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
85.0,9/16/2023,24.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,25.0,2.0,26.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
89.0,9/9/2023,25.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,15.0,2.0,25.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
92.0,9/2/2023,15.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,14.0,2.0,24.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
97.0,8/26/2023,14.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,13.0,2.0,23.0,Midnights (The Til Dawn Edition),9.506064869386669e-05,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN2

In [98]:
bill.loc[condition, ['song_release_date','track', 'acou', 'dance', 
                        'ener','instru','live', 'loud', 
                        'speech','temp', 'val', 'pop',
                         'dur', 'days', 'tot_streams',
                         'streams_n']] = new_karma_ice_values.values()

In [99]:
bill_karma_2 = bill.loc[bill['song_id'].isin(["MIDN11", "MIDN23"])]

In [100]:
bill_karma_2

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
73.0,10/14/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,31.0,2.0,30.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
76.0,10/7/2023,31.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,38.0,2.0,29.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
80.0,9/30/2023,38.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,36.0,2.0,28.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
82.0,9/23/2023,36.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,24.0,2.0,27.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
85.0,9/16/2023,24.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,25.0,2.0,26.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
89.0,9/9/2023,25.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,15.0,2.0,25.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
92.0,9/2/2023,15.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,14.0,2.0,24.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
97.0,8/26/2023,14.0,Karma (feat. Ice Spice),Taylor Swift Featuring Ice Spice,13.0,2.0,23.0,Midnights (The Til Dawn Edition),0.000095,23.0,0.0623,0.615,0.616,0.0,0.579,-6.523,0.0622,89.977,0.0734,75.0,201493.0,MIDN,MIDN23,187.0,67922508.0,11/9/2023,363221.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,Octo

### Summary of updates 
Successfully updated: 
- all song characteristics of Karma Featuring Ice Spice (MIDN22) in Billboards
- Top song to no and Top song flag to False for RED30, 1989TV22, SPEAKTV18


In [101]:
df_merged.columns

Index(['song_title', 'album_title', 'song_release_date', 'track', 'acou',
       'dance', 'ener', 'instru', 'live', 'loud', 'speech', 'temp', 'val',
       'pop', 'dur', 'album_id', 'song_id', 'days', 'tot_streams',
       'last_stream_update', 'streams_n', 'top_song', 'top_song_flag',
       'top_s_cnt', 'tv', 'alb_num', 'album_announce_mo', 'alb_ann_day',
       'alb_ann_yr', 'album_announce_date', 'album_release_mo', 'alb_r_day',
       'alb_r_yr', 'album_release_date', 'w1_sales', 'top_alb_count',
       'top_alb_normal', 'alb_track_cnt', 'tvflag', 'sales_group',
       'tot_streams_group', 'alb_genre', 'cluster', 'cnt_chart_wks', '_merge'],
      dtype='object')

In [102]:
df['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [103]:
df_merged['top_song'].value_counts(dropna=False)

top_song
top song    213
not top      69
Name: count, dtype: int64

In [104]:
df_merged['top_song_flag'].value_counts(dropna=False)

top_song_flag
True     212
False     70
Name: count, dtype: int64

In [105]:
df_merged['_merge'].value_counts(dropna=False)

_merge
both          214
left_only      68
right_only      0
Name: count, dtype: int64

In [106]:
df_merged['cnt_chart_wks'].value_counts(dropna=False)

cnt_chart_wks
1.0     98
NaN     68
2.0     32
3.0     15
20.0    11
5.0      8
4.0      5
8.0      5
6.0      4
22.0     3
19.0     3
15.0     3
13.0     2
21.0     2
36.0     2
50.0     2
7.0      2
10.0     1
23.0     1
12.0     1
49.0     1
28.0     1
25.0     1
24.0     1
14.0     1
32.0     1
38.0     1
35.0     1
33.0     1
11.0     1
9.0      1
53.0     1
29.0     1
48.0     1
Name: count, dtype: int64

## Ready to fill the 72 "NaN" values with 0 for count of chart weeks

In [107]:
df_merged['cnt_chart_wks'].fillna(0, inplace=True)

In [108]:
df_merged['cnt_chart_wks'].value_counts(dropna=False)

cnt_chart_wks
1.0     98
0.0     68
2.0     32
3.0     15
20.0    11
5.0      8
4.0      5
8.0      5
6.0      4
22.0     3
19.0     3
15.0     3
13.0     2
21.0     2
36.0     2
50.0     2
7.0      2
10.0     1
23.0     1
12.0     1
49.0     1
28.0     1
25.0     1
24.0     1
14.0     1
32.0     1
38.0     1
35.0     1
33.0     1
11.0     1
9.0      1
53.0     1
29.0     1
48.0     1
Name: count, dtype: int64

In [109]:
# Check data type of count

df_merged['cnt_chart_wks'].dtype

dtype('float64')

### Review updated dfs

In [110]:
df.shape

(282, 43)

In [111]:
df.head()

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster
0,Welcome To New York (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,1,0.009420,0.757,0.610,0.000037,0.3670,-4.840,0.0327,116.998,0.685,80,212600,1989TV,1989TV01,13,35521714,11/9/2023,2732439,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
1,Blank Space (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,2,0.088500,0.733,0.733,0.000000,0.1680,-5.376,0.0670,96.057,0.701,80,231833,1989TV,1989TV02,13,40958068,11/9/2023,3150620,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
2,Style (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,3,0.000421,0.511,0.822,0.019700,0.0899,-4.785,0.0397,94.868,0.305,81,231000,1989TV,1989TV03,13,47687102,11/9/2023,3668238,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
3,Out Of The Woods (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,4,0.000537,0.545,0.885,0.000056,0.3850,-5.968,0.0447,92.021,0.206,80,235800,1989TV,1989TV04,13,37545128,11/9/2023,2888086,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2
4,All You Had To Do Was Stay (Taylor's Version),1989 (Taylor's Version)[Deluxe],10/27/2023,5,0.000656,0.588,0.721,0.000000,0.1310,-5.579,0.0317,96.997,0.520,79,193289,1989TV,1989TV05,13,31008534,11/9/2023,2385271,top song,True,21,Taylor's Version,14,August,9,2023,8/9/2023,October,27,2023,10/27/2023,1359000,1,0.076923,22,True,High first week sales,Low total streams,Synth/Dream/Dance pop,2


In [112]:
df_merged.shape

(282, 45)

In [113]:
df_merged.tail()

,song_title,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,cluster,cnt_chart_wks,_merge
277,Our Song,Debut,10/24/2006,11,0.11100,0.668,0.672,0.000000,0.3290,-4.931,0.0303,89.011,0.539,77,201106,DEBUT,DEBUT11,6225,280561783,11/9/2023,45070,top song,True,5,original,1,June,19,2006,6/19/2006,October,24,2006,10/24/2006,39000,284,0.045622,15,False,Low first week sales,High total streams,Country pop,3,36.0,both
278,I'm Only Me When I'm With You,Debut,10/24/2006,12,0.00452,0.563,0.934,0.000807,0.1030,-3.629,0.0646,143.964,0.518,62,213053,DEBUT,DEBUT12,6225,43132276,11/9/2023,6928,not top,False,5,original,1,June,19,2006,6/19/2006,October,24,2006,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3,0.0,left_only
279,Invisible,Debut,10/24/2006,13,0.63700,0.612,0.394,0.000000,0.1470,-5.723,0.0243,96.001,0.233,58,203226,DEBUT,DEBUT13,6225,29681361,11/9/2023,4768,not top,False,5,original,1,June,19,2006,6/19/2006,October,24,2006,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3,0.0,left_only
280,A Perfectly Good Heart,Debut,10/24/2006,14,0.00349,0.483,0.751,0.000000,0.1280,-5.726,0.0365,156.092,0.268,57,220146,DEBUT,DEBUT14,6225,21365584,11/9/2023,3432,not top,False,5,original,1,June,19,2006,6/19/2006,October,24,2006,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3,0.0,left_only
281,Teardrops on My Guitar - Pop Version,Debut,10/24/2006,15,0.04020,0.459,0.753,0.000000,0.0863,-3.827,0.0537,199.997,0.483,57,179066,DEBUT,DEBUT15,6225,28224285,11/9/2023,4534,not top,False,5,original,1,June,19,2006,6/19/2006,October,24,2006,10/24/2006,39000,284,0.045622,15,False,Low first week sales,Low total streams,Country pop,3,0.0,left_only


In [114]:
df_merged.isnull().sum()

song_title             0
album_title            0
song_release_date      0
track                  0
acou                   0
dance                  0
ener                   0
instru                 0
live                   0
loud                   0
speech                 0
temp                   0
val                    0
pop                    0
dur                    0
album_id               0
song_id                0
days                   0
tot_streams            0
last_stream_update     0
streams_n              0
top_song               0
top_song_flag          0
top_s_cnt              0
tv                     0
alb_num                0
album_announce_mo      0
alb_ann_day            0
alb_ann_yr             0
album_announce_date    0
album_release_mo       0
alb_r_day              0
alb_r_yr               0
album_release_date     0
w1_sales               0
top_alb_count          0
top_alb_normal         0
alb_track_cnt          0
tvflag                 0
sales_group            0


In [115]:
bill.shape

(1446, 49)

In [116]:
bill.tail(10)

,chart_week,chart_position,song_title,artist,prior_wk_position,peak_pos,wks_on_chart,album_title,song_release_date,track,acou,dance,ener,instru,live,loud,speech,temp,val,pop,dur,album_id,song_id,days,tot_streams,last_stream_update,streams_n,top_song,top_song_flag,top_s_cnt,tv,alb_num,album_announce_mo,alb_ann_day,alb_ann_yr,album_announce_date,album_release_mo,alb_r_day,alb_r_yr,album_release_date,w1_sales,top_alb_count,top_alb_normal,alb_track_cnt,tvflag,sales_group,tot_streams_group,alb_genre,_merge
1245.0,3/28/2009,40.0,You're Not Sorry,Taylor Swift,67.0,11.0,4.0,Fearless Platinum Edition,11/11/2008,15.0,0.130,0.459,0.459,0.000002,0.113,-4.126,0.0275,134.018,0.281,44.0,261800.0,FEAR,FEAR15,5476.0,444440461.0,11/9/2023,81161.0,top song,True,2.0,original,2.0,September,15.0,2008.0,9/15/2008,November,11.0,2008.0,11/11/2008,592300.0,261.0,0.047663,19.0,False,Medium first week sales,High total streams,Country pop,both
1249.0,3/21/2009,67.0,You're Not Sorry,Taylor Swift,0.0,11.0,3.0,Fearless Platinum Edition,11/11/2008,15.0,0.130,0.459,0.459,0.000002,0.113,-4.126,0.0275,134.018,0.281,44.0,261800.0,FEAR,FEAR15,5476.0,444440461.0,11/9/2023,81161.0,top song,True,2.0,original,2.0,September,15.0,2008.0,9/15/2008,November,11.0,2008.0,11/11/2008,592300.0,261.0,0.047663,19.0,False,Medium first week sales,High total streams,Country pop,both
1288.0,11/22/2008,35.0,You're Not Sorry,Taylor Swift,11.0,11.0,2.0,Fearless Platinum Edition,11/11/2008,15.0,0.130,0.459,0.459,0.000002,0.113,-4.126,0.0275,134.018,0.281,44.0,261800.0,FEAR,FEAR15,5476.0,444440461.0,11/9/2023,81161.0,top song,True,2.0,original,2.0,September,15.0,2008.0,9/15/2008,November,11.0,2008.0,11/11/2008,592300.0,261.0,0.047663,19.0,False,Medium first week sales,High total streams,Country pop,both
1292.0,11/15/2008,11.0,You're Not Sorry,Taylor Swift,0.0,11.0,1.0,Fearless Platinum Edition,11/11/2008,15.0,0.130,0.459,0.459,0.000002,0.113,-4.126,0.0275,134.018,0.281,44.0,261800.0,FEAR,FEAR15,5476.0,444440461.0,11/9/2023,81161.0,top song,True,2.0,original,2.0,September,15.0,2008.0,9/15/2008,November,11.0,2008.0,11/11/2008,592300.0,261.0,0.047663,19.0,False,Medium first week sales,High total streams,Country pop,both
232.0,12/10/2022,80.0,"You're On Your Own, Kid",Taylor Swift,67.0,8.0,6.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,0.423,0.694,0.386,0.000010,0.126,-10.312,0.0597,120.044,0.390,70.0,194206.0,MIDN,MIDN05,384.0,437706786.0,11/9/2023,1139861.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
251.0,12/3/2022,67.0,"You're On Your Own, Kid",Taylor Swift,48.0,8.0,5.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,0.423,0.694,0.386,0.000010,0.126,-10.312,0.0597,120.044,0.390,70.0,194206.0,MIDN,MIDN05,384.0,437706786.0,11/9/2023,1139861.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
259.0,11/26/2022,48.0,"You're On Your Own, Kid",Taylor Swift,42.0,8.0,4.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,0.423,0.694,0.386,0.000010,0.126,-10.312,0.0597,120.044,0.390,70.0,194206.0,MIDN,MIDN05,384.0,437706786.0,11/9/2023,1139861.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
273.0,11/19/2022,42.0,"You're On Your Own, Kid",Taylor Swift,20.0,8.0,3.0,Midnights (The Til Dawn Edition),10/21/2022,5.0,0.423,0.694,0.386,0.000010,0.126,-10.312,0.0597,120.044,0.390,70.0,194206.0,MIDN,MIDN05,384.0,437706786.0,11/9/2023,1139861.0,top song,True,12.0,original,12.0,August,28.0,2022.0,8/28/2022,October,21.0,2022.0,10/21/2022,1140000.0,54.0,0.140625,23.0,False,High first week sales,High total streams,Synth/Dream/Dance pop,both
297.0,11/12/2022,20.0,"You're 

In [117]:
bill.isnull().sum()

chart_week             0
chart_position         0
song_title             0
artist                 0
prior_wk_position      0
peak_pos               0
wks_on_chart           0
album_title            0
song_release_date      0
track                  0
acou                   0
dance                  0
ener                   0
instru                 0
live                   0
loud                   0
speech                 0
temp                   0
val                    0
pop                    0
dur                    0
album_id               0
song_id                0
days                   0
tot_streams            0
last_stream_update     0
streams_n              0
top_song               0
top_song_flag          0
top_s_cnt              0
tv                     0
alb_num                0
album_announce_mo      0
alb_ann_day            0
alb_ann_yr             0
album_announce_date    0
album_release_mo       0
alb_r_day              0
alb_r_yr               0
album_release_date     0


In [118]:
# Drop NaN rows

bill.dropna(inplace= True)

In [119]:
bill.shape

(1446, 49)

# 04. Data Export

In [120]:
# Export songs dataframe with updates to top song columns 
# Using the same name since edits were errors vs. new values
# Can update all visualizations accordingly

df.to_csv(os.path.join(path, '02 Manipulated Data', 'song_stream_alb_final_3.csv'))

In [121]:
# Export billboard dataframe with updates to song id
# Using the same name since edits were errors vs. new values
# Can update all visualizations accordingly

bill.to_csv(os.path.join(path, '02 Manipulated Data', 'df_hot_100_ts_only_dec_23.csv'))

In [122]:
## Export the merged songs dataframe
# This dataframe will be used for cluster analysis

df_merged.to_csv(os.path.join(path, '02 Manipulated Data', 'song_stream_alb_final_4.csv'))